In [2]:
import numpy as np
import pandas as pd
import gensim, logging
import gensim.downloader as api
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA, TruncatedSVD

from utils import classification_metrics

In [ ]:
model = api.load("word2vec-google-news-300", return_path=True)

[--------------------------------------------------] 1.6% 26.2/1662.8MB downloaded

In [ ]:
train = pd.read_csv('./data/processed_data.csv')

train = train[['X', 'Output']]
train = train.dropna()

In [ ]:
model.wv.__getitem__('series')

In [ ]:
temp = []
for _, document in train.iterrows():
    document_embedding = []
    for word in document['X'].split(' '):
        try:
            word_embedding = model.wv.__getitem__(word)
            document_embedding.append((word_embedding))
        except:
            pass
    temp.append([document['X'], document_embedding, document['Output']])

In [ ]:
data = pd.DataFrame(temp, columns=['X', 'embeddings', 'y'])

In [ ]:
data['len'] = data['embeddings'].apply(len)

In [ ]:
data = data[data['len']>0]

In [ ]:
data['processed'] = list(map(lambda x: np.array(x).mean(0), data['embeddings']))

In [ ]:
x = []
for i in data['processed']:
    x.append(i)

In [ ]:
x = np.array(x)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(x, data['y'], test_size=0.2, random_state=42)
print([np.shape(X_train), np.shape(X_test)])

In [ ]:
classifier = LogisticRegression(penalty='l1', solver='liblinear')
parameters = {'C':[0.1, 1, 10, 50, 100, 200, 400]}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'neg_log_loss',
                           cv = 3)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_cross_entropy_loss = grid_search.best_score_

In [ ]:
best_cross_entropy_loss, best_parameters

In [ ]:
lg = LogisticRegression(penalty='l1', C=10)
lg.fit(X_train, y_train)

In [ ]:
classification_metrics(y_train, lg.predict(X_train), msg = 'Train')

In [ ]:
classification_metrics(y_test, lg.predict(X_test), msg = 'Test')